In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
# https://discuss.pytorch.org/t/how-to-implement-keras-layers-core-lambda-in-pytorch/5903
class BestMeanPooling(nn.Module):
    
  def __init__(self, n_topN):
    super(BestMeanPooling, self).__init__()
    self.topN = n_topN
        
  def forward(self, aX_tr):
    """
    
    """
    aX_tr_sorted, _ = torch.sort(aX_tr, axis=1) # also return index
    return torch.mean(aX_tr_sorted[:, -self.topN:,:], axis=1)
    # print("B = ", aX_tr_sorted.size(), aX_tr_sorted)
    # c = torch.mean(aX_tr_sorted[:, -self.topN:,:], axis=1)
    # print("C = ", c.size(), c)
    return c

# Test
aA = torch.randn(1, 200).reshape(4, 10, 5)
mean_pool = BestMeanPooling(2) 
aA = mean_pool(aA)
# print(aA)

In [3]:
class SyntheticData(Dataset):
    def __init__(self, upper, x, y, z):
      self.aX_tr_sy = np.random.random_sample(upper).reshape(x, y, z)
      self.ay_tr_sy = np.random.randint(0, 2, x)

    def __len__(self):
        return len(self.aX_tr_sy)

    def __getitem__(self, idx):
        return self.aX_tr_sy[idx], self.ay_tr_sy[idx] #.squeeze(1)

In [4]:
class MassCNN(nn.Module):

  def __init__(self, n_in, n_out, num_event=10, topN=5, prob=0.00):
    super(MassCNN,self).__init__()
    self.n_filter = n_out

    #first layers
    self.conv_1        = nn.Conv1d(n_in, n_out, kernel_size=1)
    
    # output of the layer is # of filters
    self.bmpool_1      = BestMeanPooling(topN)
    self.dropout_L1    = nn.Dropout(p=prob)
    
    # Fully connected layer
    self.fcn_1 = nn.Linear(3, 2)
    # self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, aX_tr):
    aX_tr = self.conv_1(aX_tr)
    aX_tr = torch.relu(aX_tr)
    
    # This will collapse one dimension
    aX_tr = self.bmpool_1(aX_tr)
    
    if self.n_filter > 4: 
      aX_tr = self.dropout_L1(aX_tr)
    
    aX_tr = aX_tr.view(aX_tr.size(0), -1)
    aX_tr = self.fcn_1(aX_tr)
    # aX_tr = self.softmax(aX_tr)
    return aX_tr

In [5]:
def train(n_epochs=10, lr=0.1, filter=12, maxpool_percent=100.00, n_iter = 0,
          check_point=False):
    """
      Train the model with Monte Carlo sampling in which hyperparameters
      are randomly generated. Multiple simulations are run to obtain 
      statistically robust models.

      params:

      returns:
        # of best solutions (models)
    """
    # Random data
    d_trains = SyntheticData(750, 5, 50, 3)
    d_valids = SyntheticData(750, 5, 50, 3)
   
    t_cost_per_epoch = []
    
    # model.state_dict()['cnn1.weight']
    # model.state_dict()['cnn2.weight']
    af_Tr = DataLoader(d_trains, batch_size=100, shuffle=True)
    af_Va = DataLoader(d_valids, batch_size=10, shuffle=True)
    t_solutions = []
    for epoch in range(n_epochs):
      # model, optimizer, and lost function
      o_mass_model = MassCNN(50, filter, 3, 5)
      o_optimizer = torch.optim.SGD(o_mass_model.parameters(), lr=0.0001)
      o_cost      = torch.nn.CrossEntropyLoss()
      m_models = dict()
      fgr_cost = 0
      for af_X, af_y in af_Tr:  # iterative over batch of data
        o_optimizer.zero_grad() ## Back propagation the loss
        af_y_pr  = o_mass_model(af_X.float())
        o_loss   = o_cost(af_y_pr, af_y)
        o_loss.backward()
        o_optimizer.step() # back propagate
        fgr_cost += o_loss.data
      t_cost_per_epoch.append(fgr_cost)
    
      #print(o_mass_model.state_dict().keys())
      #print("Epochs in fun ", epoch, id(o_mass_model))
      # Where is validation loss?
      o_mass_model.eval()
      f_correct = 0
      for af_x_va, af_y_va in af_Va:
        z  = o_mass_model(af_x_va.float()).data
        _, yhat = torch.max(z.data, 1)
        f_correct += (yhat == af_y_va).sum().item()
      f_accuracy = "%0.3f" %(f_correct / 1000) # fixed this with sample size
    
      m_models["epoch"] = epoch 
      m_models["iteration"] = n_iter
      m_models["val_acc"] = f_accuracy
      m_models["model_state"] =  o_mass_model.state_dict()
      m_models["optimizer_state"] =  o_optimizer.state_dict()
      m_models["loss"] =  o_cost.state_dict()
      m_models["model"] = o_mass_model
      t_solutions.append(m_models.copy())
      del m_models
    return t_solutions

In [6]:
def get_best_models(t_models):
  """
  Select the best solutions from the pool
  
  params: 
    t_models: list of models. Each model is dictionary containing parameters
              and hyperparameters.
  return: 
    list of best models
  """
  n_best_sol      = 5
  t_best_val_accs = []
  t_best_models   = []
    
  for o_model in t_models: 
    b_update  = True
    f_val_acc = float(o_model["val_acc"])
    if len(t_best_val_accs) >= n_best_sol:
      print(t_best_val_accs)
      f_lowest_acc = np.min(t_best_val_accs)
      if f_val_acc > f_lowest_acc:
        i_cur_idx = t_best_val_accs.index(f_lowest_acc)
        t_best_val_accs.pop(i_cur_idx)
        t_best_models.pop(i_cur_idx)
      else:
        b_update = False
    if b_update: 
      t_best_val_accs.append(f_val_acc)
      t_best_models.append(o_model)
  return t_best_models

In [7]:
# load the multiple models for testing

def test_model(d_tests, t_best_sols):
  """
  Test performance of model on independent data. Since multiple models were generated, 
  these models were tested aganist an independent data and their average performance 
  will be returned as final result.
  
  params:
    d_tests: data for independent test
    t_best_sols: list of best solutions
  """
  t_test_accs = []
  for o_best_sol in t_best_sols:
    o_best_model = o_best_sol["model"]
    o_best_model.eval()
    f_correct = 0
    n_test_count = len(d_tests)
    for d_test_X, d_target_Y in d_tests:
      z  = o_best_model(d_test_X.float()).data # remove float
      _, yhat = torch.max(z.data, 1)
      f_correct += (yhat == d_target_Y).sum().item()
    f_accuracy = "%0.3f" %(f_correct / n_test_count) # fixed this with sample size
    t_test_accs.append(float(f_accuracy))
  print("Coding: ", t_test_accs)
  return np.mean(t_test_accs)

# model.load_state_dict(checkpoint['model_state_dict'])

In [8]:
# get 20% of best solutions after iterating 
# between 15 to 30 times (these # are randomly selected)

n_cell = 10
f_maxpools    = [0.01, 1.0, 5.0, 20.0, 100.0]
n_maxpool_len = len(f_maxpools)

t_models      = []
for n_trial in range(10):
  f_lr       = 10 ** np.random.uniform(-3, -2)
  i_filter   = np.random.choice(range(3,10))
  f_max_pool = f_maxpools[n_trial%n_maxpool_len]
  f_maxpool  = max(1, int(f_max_pool/100. * n_cell))
  t_models  += train(10, f_lr, i_filter, f_maxpool)
  # if b_checkpoint: # save check point files
    
t_best_models = get_best_models(t_models)

# generate test data
d_tests  = SyntheticData(750, 5, 50, 3)
af_test  = DataLoader(d_tests)
predicted_Y = test_model(af_test, t_best_models)
print("Coding: ", predicted_Y)

[0.002, 0.002, 0.002, 0.002, 0.002]
[0.002, 0.002, 0.002, 0.002, 0.003]
[0.002, 0.002, 0.002, 0.003, 0.003]
[0.002, 0.002, 0.003, 0.003, 0.003]
[0.002, 0.002, 0.003, 0.003, 0.003]
[0.002, 0.003, 0.003, 0.003, 0.003]
[0.002, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003,

### Keras

In [9]:
from keras.layers import Input, Dense, Lambda, Activation, Dropout
from keras.layers.convolutional import Convolution1D
from keras.models import Model
from keras.optimizers import Adam

# the input layer
ncell = 20
nmark = 5
data_input = Input(shape=(ncell, nmark))

nfilter = 10
# the filters
print(data_input.shape)
conv = Convolution1D(nfilter, 1, activation='linear', name='conv1')(data_input)
print(conv.shape)


import keras.backend as K
K.print_tensor(conv)

Using TensorFlow backend.


(None, 20, 5)
(None, 20, 10)


<tf.Tensor 'Identity:0' shape=(None, 20, 10) dtype=float32>

In [10]:
a = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=7, stride=1, padding=3)

In [11]:
"""
in_channel = 16
out_channel = 33
filter/kernal = 3
batch = 20
"""

m = nn.Conv1d(3, 6, 2, stride=1)
input = torch.randn(5, 3, 4)
print(input.shape)
output = m(input)
print(output.shape)

torch.Size([5, 3, 4])
torch.Size([5, 6, 3])


In [12]:
output.shape

torch.Size([5, 6, 3])

In [13]:
input

tensor([[[ 1.0291e+00, -3.4732e-01, -1.9905e+00,  2.9125e-01],
         [-1.6340e+00,  2.1681e-01,  7.8561e-01, -6.0226e-01],
         [ 8.7316e-01, -2.3531e-01, -1.0035e+00,  4.2327e-01]],

        [[-5.8480e-01, -5.1901e-01,  1.3230e-01,  1.5052e+00],
         [ 8.2028e-01,  1.4783e+00, -9.7886e-01,  4.1739e-01],
         [-3.1428e-01, -8.5926e-01, -2.6044e-01,  7.8873e-01]],

        [[ 1.2651e+00,  8.4648e-01, -9.4190e-01,  6.6988e-01],
         [ 7.3282e-01,  1.4394e-01,  4.8916e-04,  1.6228e+00],
         [-1.1017e+00, -3.8403e-01, -1.1204e-01,  2.3949e-01]],

        [[-1.3718e+00, -1.4614e+00, -8.7649e-02,  1.8829e+00],
         [-5.6087e-01,  1.1582e+00, -6.7663e-01, -1.1620e+00],
         [-5.0400e-01, -6.1528e-01, -8.3913e-01,  3.6212e-01]],

        [[-1.3789e+00,  4.6552e-01, -4.7711e-01,  2.3987e-03],
         [ 2.2687e+00,  3.3071e-01, -6.9713e-01, -6.7381e-01],
         [-8.1790e-01, -2.4145e+00, -9.2545e-01, -1.6154e+00]]])

In [14]:
output

tensor([[[ 0.3641,  0.7343,  0.3518],
         [ 0.5452, -0.3405,  0.2656],
         [ 0.2508,  0.1945, -0.6212],
         [ 0.8256,  0.9600,  0.2388],
         [ 0.0887, -0.1202,  0.8158],
         [ 0.6137,  0.2650, -0.1434]],

        [[ 0.5647, -0.6174,  0.2952],
         [-0.2418, -0.1800,  0.4797],
         [ 0.2544, -0.7770, -0.1015],
         [ 0.3122,  0.2029, -0.2616],
         [ 0.1012,  0.6687,  0.5326],
         [-0.3271,  0.2342, -0.0328]],

        [[-0.7463, -0.1336,  0.8619],
         [-0.4816, -0.5939,  0.1808],
         [-0.5614, -0.0674,  0.4098],
         [-0.4515,  0.3491, -0.0699],
         [ 0.3950, -0.0567,  0.2680],
         [ 0.2312,  0.5782, -0.5729]],

        [[ 1.5025, -0.0068, -0.2961],
         [ 0.1962,  0.4127,  0.9048],
         [ 0.1880, -0.7258, -1.0428],
         [ 0.8816,  0.6617, -0.0872],
         [ 0.1281,  0.8463,  1.1389],
         [-0.0788,  0.0059,  0.2821]],

        [[-0.2409, -0.2157,  0.3521],
         [ 0.4806, -0.4048,  1.0856],
    

In [15]:
from heapq import heappush, heappop
heap = []
data = [(1, 'J'), (4, 'N'), (3, 'H'), (2, 'O')]
for item in data:
  heappush(heap, item)
  print("item ", item, "heap ", heap)

while heap:
  print(heappop(heap)[1])

item  (1, 'J') heap  [(1, 'J')]
item  (4, 'N') heap  [(1, 'J'), (4, 'N')]
item  (3, 'H') heap  [(1, 'J'), (4, 'N'), (3, 'H')]
item  (2, 'O') heap  [(1, 'J'), (2, 'O'), (3, 'H'), (4, 'N')]
J
O
H
N


In [16]:
import numpy as np
a = list(np.random.uniform(10, 15, 10))
np.min(a)

10.391674201191805

In [17]:
a

[10.391674201191805,
 12.592464370966416,
 14.611942123977197,
 14.34392951653233,
 12.02532732428294,
 10.543753908281914,
 13.859416915889499,
 10.786223147420115,
 14.640421434377895,
 12.629468290152852]